In [18]:
from keras.preprocessing.image import ImageDataGenerator


from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import *
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications import ResNet50

import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score

In [2]:
imagePath = './dataSet/splitImages/'

### 이미지 데이터 생성

ImageDataGenerator (train, validation)

In [4]:
trainDataGen = ImageDataGenerator(rescale=1./255,
                                  rotation_range = 30,
                                  width_shift_range=0.2,
                                  height_shift_range=0.1,
                                  shear_range=30,
                                  zoom_range=0.4,
                                  horizontal_flip=False,
                                  vertical_flip=False,
                                  brightness_range=[0.7, 1.3],
                                  fill_mode='nearest',
                                  validation_split=.1,
                                  )
batchSize = 100
imageWidth = 224
imageHeight = 224
imageChannel = 3

train set

In [7]:
trainGenSet = trainDataGen.flow_from_directory(
    imagePath + 'train',
    batch_size=batchSize,
    target_size=(imageWidth,imageHeight),
    class_mode='categorical',
    subset='training'
)

Found 5062 images belonging to 44 classes.


validation set

In [9]:
validationGenSet = trainDataGen.flow_from_directory(
    imagePath + 'train',
    batch_size=batchSize,
    target_size=(imageWidth,imageHeight),
    class_mode='categorical',
    subset='validation'
)

Found 539 images belonging to 44 classes.


ImageDataGenerator (train, validation)

In [10]:
testDataGen = ImageDataGenerator(rescale=1./255)

test set

In [11]:
testGenSet = testDataGen.flow_from_directory(
    imagePath + 'test',
    batch_size=batchSize,
    target_size=(imageWidth,imageHeight),
    class_mode='categorical'
)

Found 739 images belonging to 44 classes.


In [13]:
model = Sequential()
model.add(ResNet50(include_top=True, weights=None, input_shape=(imageWidth, imageHeight, imageChannel), classes=44))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 44)                23677868  
                                                                 
Total params: 23,677,868
Trainable params: 23,624,748
Non-trainable params: 53,120
_________________________________________________________________


In [14]:
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [15]:
checkpointPath = './checkpoints/epoch_{epoch:03d}.ckpt'

In [16]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(
                        checkpointPath, monitor='val_loss', verbose=0, save_best_only=True,
                        save_weights_only=True, mode='auto', save_freq='epoch'
                    )


In [17]:
earlyStopping = EarlyStopping(monitor='val_loss',
                              patience=5, 
                             )

In [ ]:
epochs = 32
history = model.fit_generator(
    trainGenSet,
    epochs=epochs,
    steps_per_epoch=trainGenSet.samples / epochs,
    validation_data=validationGenSet,
    validation_steps=validationGenSet.samples / epochs,
    callbacks=[checkpoint,earlyStopping]
)

In [ ]:
scores = model.evaluate_generator(testGenSet)
print(scores)

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, accuracy, label='Training Accuracy')
plt.plot(epochs_range, val_accuracy, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
model.save_weights('./models/resnetModel')